<a href="https://colab.research.google.com/github/hadwin-357/GCN/blob/main/Graph_level_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Install required packages.
import os
import torch
os.environ['TORCH'] = torch.__version__
print(torch.__version__)

!pip install -q torch-scatter -f https://data.pyg.org/whl/torch-${TORCH}.html
!pip install -q torch-sparse -f https://data.pyg.org/whl/torch-${TORCH}.html
!pip install -q git+https://github.com/pyg-team/pytorch_geometric.git

2.1.0+cu121
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.8/10.8 MB 76.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.0/5.0 MB 34.7 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [3]:
#Graph classification: need to distinguish differnt graph:
# use molecular property prediction as a example
#data from torch_geometric.datasets.TUDataset (from TU Dortmund University)

import torch
from torch_geometric.datasets import TUDataset

dataset = TUDataset(root='data/TUDataset', name='MUTAG')

#GET familar with the data
print (f'Dataset:{dataset}')
print(f'Num of graphes:{len(dataset)}')
print(f'Num of features:{dataset.num_features}')
print(f'Num of classes:{dataset.num_classes}')

#get the first data
data= dataset[0]

print(f'First data: {data}')
print(f'Num of Nodes:{data.num_nodes}')
print(f'Num of Edges:{data.num_edges}')
print(f'Average Node degrees:{data.num_edges/data.num_nodes: 2f}')
print(f'Has isolated nodes:{data.has_isolated_nodes()}')
print(f'Has self-loops:{data.has_self_loops()}')
print(f'is directional:{data.is_directed()}')

Dataset:MUTAG(188)
Num of graphes:188
Num of features:7
Num of classes:2
First data: Data(edge_index=[2, 38], x=[17, 7], edge_attr=[38, 4], y=[1])
Num of Nodes:17
Num of Edges:38
Average Node degrees: 2.235294
Has isolated nodes:False
Has self-loops:False
is directional:False


Processing...
Done!


In [4]:
# shuffle the data and split the data into train and test
torch.manual_seed(42)
dataset  = dataset.shuffle()

train_dataset =dataset[:150]
test_dataset =dataset[150:]

print(f'num of training graphs:{train_dataset} num of testing graphs:{test_dataset}')

num of training graphs:MUTAG(150) num of testing graphs:MUTAG(38)


In [5]:
# batching in graph learning
from torch_geometric.loader import DataLoader
train_dataloader = DataLoader(train_dataset, batch_size=32, shuffle =True)
test_dataloader = DataLoader(test_dataset, batch_size=32, shuffle=False)

for i , data in enumerate(train_dataloader):
  print(f'Bath :{i+1}')
  print(f'Num of graphes in this batch:{data.num_graphs}')
  print(data)

Bath :1
Num of graphes in this batch:32
DataBatch(edge_index=[2, 1202], x=[550, 7], edge_attr=[1202, 4], y=[32], batch=[550], ptr=[33])
Bath :2
Num of graphes in this batch:32
DataBatch(edge_index=[2, 1316], x=[589, 7], edge_attr=[1316, 4], y=[32], batch=[589], ptr=[33])
Bath :3
Num of graphes in this batch:32
DataBatch(edge_index=[2, 1266], x=[569, 7], edge_attr=[1266, 4], y=[32], batch=[569], ptr=[33])
Bath :4
Num of graphes in this batch:32
DataBatch(edge_index=[2, 1252], x=[574, 7], edge_attr=[1252, 4], y=[32], batch=[574], ptr=[33])
Bath :5
Num of graphes in this batch:22
DataBatch(edge_index=[2, 940], x=[421, 7], edge_attr=[940, 4], y=[22], batch=[421], ptr=[23])


AttributeError: 'TUDataset' object has no attribute 'num_graphs'

In [20]:
from torch_geometric.nn import GCNConv
from torch_geometric.nn.pool import global_add_pool
from torch.nn import Linear
import torch.nn.functional as F

# model_0, edge features are not used
class GCN(torch.nn.Module):
  def __init__(self, hidden_channels):
    super().__init__()
    torch.manual_seed(32)
    self.conv1 = GCNConv(in_channels=dataset.num_features, out_channels=hidden_channels)
    self.conv2 = GCNConv(in_channels=hidden_channels, out_channels=hidden_channels)
    self.conv3 = GCNConv(in_channels=hidden_channels, out_channels=hidden_channels)
    self.lin = Linear(in_features=hidden_channels, out_features=dataset.num_classes)

  def forward(self, x, edge_index, batch):
    x = self.conv1(x, edge_index)
    x = x.relu()
    x = self.conv2(x, edge_index)
    x = x.relu()
    x = self.conv3(x, edge_index)

    # readout layer using Mean aggregation
    x = global_add_pool(x, batch) # batch info is needed to calculate the size of each sample in the batch)

    # final classifier layer
    x = F.dropout(x, p=0.5, training=self.training)
    x = self.lin(x)

    return x

model_0 = GCN(hidden_channels=64)
print (model_0)



GCN(
  (conv1): GCNConv(7, 64)
  (conv2): GCNConv(64, 64)
  (conv3): GCNConv(64, 64)
  (lin): Linear(in_features=64, out_features=2, bias=True)
)


In [17]:
from IPython.display import Javascript
display(Javascript('''google.colab.output.setIframeHeight(0, true, {maxHeight: 300})'''))

import torch_geometric

def train(model: torch.nn.Module,
          dataloader: torch_geometric.loader.DataLoader,
          loss_fn: torch.nn.Module,
          optimizer: torch.optim.Optimizer):
  train_loss, train_acc = 0, 0
  model.train()
  for data in dataloader:
    out = model(data.x, data.edge_index, data.batch)
    loss =loss_fn(out, data.y)
    train_loss +=loss
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    train_acc +=(out.argmax(dim=1)==data.y).sum()/len(data.y)

  #normalize the train_loss and tran_acc over batch
  train_loss = train_loss/len(dataloader)
  train_acc = train_acc/len(dataloader)

  return train_loss, train_acc

def test(model: torch.nn.Module,
          dataloader: torch_geometric.loader.DataLoader,
          loss_fn: torch.nn.Module,
          ):
  test_loss, test_acc = 0, 0
  model.eval()
  with torch.inference_mode():
    for data in dataloader:
      out = model(data.x, data.edge_index, data.batch)
      loss =loss_fn(out, data.y)
      test_loss +=loss
      test_acc += (out.argmax(dim=1)==data.y).sum()/len(data.y)

  #normalize the train_loss and tran_acc over batch
  test_loss = test_loss/len(dataloader)
  test_acc = test_acc/len(dataloader)

  return test_loss, test_acc

loss_fn = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model_0.parameters(), lr =0.01, weight_decay=1e-4)

for epoch in range(1, 200):
  train_loss, train_acc=train(model_0, train_dataloader, loss_fn=loss_fn, optimizer=optimizer)
  test_loss, test_acc = test(model_0, test_dataloader, loss_fn= loss_fn)
  print(f'Epoch: {epoch}: train_loss/test_loss: {train_loss:2f}/{test_loss:2f} train_acc/test_acc:{train_acc:2f}/{test_acc:2f}')





<IPython.core.display.Javascript object>

Epoch: 1: train_loss/test_loss: 0.432701/0.708158 train_acc/test_acc:0.779545/0.692708
Epoch: 2: train_loss/test_loss: 0.411723/0.669521 train_acc/test_acc:0.837500/0.677083
Epoch: 3: train_loss/test_loss: 0.427356/0.706128 train_acc/test_acc:0.838636/0.739583
Epoch: 4: train_loss/test_loss: 0.413721/0.731951 train_acc/test_acc:0.803977/0.661458
Epoch: 5: train_loss/test_loss: 0.342158/0.655460 train_acc/test_acc:0.860227/0.723958
Epoch: 6: train_loss/test_loss: 0.426444/0.678379 train_acc/test_acc:0.786364/0.692708
Epoch: 7: train_loss/test_loss: 0.368577/0.700707 train_acc/test_acc:0.828977/0.723958
Epoch: 8: train_loss/test_loss: 0.400333/0.743533 train_acc/test_acc:0.838636/0.677083
Epoch: 9: train_loss/test_loss: 0.359094/0.698822 train_acc/test_acc:0.825568/0.692708
Epoch: 10: train_loss/test_loss: 0.357790/0.776389 train_acc/test_acc:0.869318/0.645833
Epoch: 11: train_loss/test_loss: 0.347746/0.794220 train_acc/test_acc:0.847727/0.708333
Epoch: 12: train_loss/test_loss: 0.377924

In [22]:
#improved model with a simple skip-connection to the GNN layer to perserve the central node info

from torch_geometric.nn import GraphConv
from torch_geometric.nn.pool import global_mean_pool


class GNN(torch.nn.Module):
    def __init__(self, hidden_channels):
        super(GNN, self).__init__()
        torch.manual_seed(12345)
        self.conv1 = GraphConv(in_channels=dataset.num_features, out_channels=hidden_channels)
        self.conv2 =  GraphConv(in_channels=hidden_channels, out_channels=hidden_channels)
        self.conv3 =  GraphConv(in_channels=hidden_channels, out_channels=hidden_channels)
        self.lin = Linear(hidden_channels, dataset.num_classes)

    def forward(self, x, edge_index, batch):
        x = self.conv1(x, edge_index)
        x = x.relu()
        x = self.conv2(x, edge_index)
        x = x.relu()
        x = self.conv3(x, edge_index)

        x = global_mean_pool(x, batch)

        x = F.dropout(x, p=0.5, training=self.training)
        x = self.lin(x)

        return x

model_2 = GNN(hidden_channels=64)
print(model_2)

GNN(
  (conv1): GraphConv(7, 64)
  (conv2): GraphConv(64, 64)
  (conv3): GraphConv(64, 64)
  (lin): Linear(in_features=64, out_features=2, bias=True)
)


In [24]:
from IPython.display import Javascript
display(Javascript('''google.colab.output.setIframeHeight(0, true, {maxHeight: 300})'''))


loss_fn = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model_2.parameters(), lr =0.01)

for epoch in range(1, 200):
  train_loss, train_acc=train(model_2, train_dataloader, loss_fn=loss_fn, optimizer=optimizer)
  test_loss, test_acc = test(model_2, test_dataloader, loss_fn= loss_fn)
  print(f'Epoch: {epoch}: train_loss/test_loss: {train_loss:2f}/{test_loss:2f} train_acc/test_acc:{train_acc:2f}/{test_acc:2f}')

<IPython.core.display.Javascript object>

Epoch: 1: train_loss/test_loss: 0.193233/1.273976 train_acc/test_acc:0.934659/0.671875
Epoch: 2: train_loss/test_loss: 0.141264/0.617786 train_acc/test_acc:0.940909/0.687500
Epoch: 3: train_loss/test_loss: 0.123949/1.091620 train_acc/test_acc:0.956250/0.770833
Epoch: 4: train_loss/test_loss: 0.094563/1.092825 train_acc/test_acc:0.968750/0.770833
Epoch: 5: train_loss/test_loss: 0.143405/0.914275 train_acc/test_acc:0.940909/0.770833
Epoch: 6: train_loss/test_loss: 0.086304/1.140306 train_acc/test_acc:0.968750/0.770833
Epoch: 7: train_loss/test_loss: 0.084938/0.685625 train_acc/test_acc:0.975000/0.703125
Epoch: 8: train_loss/test_loss: 0.103977/1.363149 train_acc/test_acc:0.943750/0.786458
Epoch: 9: train_loss/test_loss: 0.092716/1.124108 train_acc/test_acc:0.956818/0.786458
Epoch: 10: train_loss/test_loss: 0.088901/1.093657 train_acc/test_acc:0.938068/0.703125
Epoch: 11: train_loss/test_loss: 0.071972/1.097287 train_acc/test_acc:0.981250/0.703125
Epoch: 12: train_loss/test_loss: 0.060015